# MPIA Arxiv on Deck 2

Contains the steps to produce the paper extractions.

In [1]:
# Imports
import os
from IPython.display import Markdown, display
from tqdm.notebook import tqdm
import warnings
from PIL import Image 

# requires arxiv_on_deck_2

from arxiv_on_deck_2.arxiv2 import (get_new_papers, 
                                    get_paper_from_identifier,
                                    retrieve_document_source, 
                                    get_markdown_badge)
from arxiv_on_deck_2 import (latex,
                             latex_bib,
                             mpia,
                             highlight_authors_in_list)

# Sometimes images are really big
Image.MAX_IMAGE_PIXELS = 1000000000 

In [2]:
# Some useful definitions.

class AffiliationWarning(UserWarning):
    pass

class AffiliationError(RuntimeError):
    pass

def validation(source: str):
    """Raises error paper during parsing of source file
    
    Allows checks before parsing TeX code.
    
    Raises AffiliationWarning
    """
    check = mpia.affiliation_verifications(source, verbose=True)
    if check is not True:
        raise AffiliationError("mpia.affiliation_verifications: " + check)

        
warnings.simplefilter('always', AffiliationWarning)


def get_markdown_qrcode(paper_id: str):
    """ Generate a qrcode to the arxiv page using qrserver.com
    
    :param paper: Arxiv paper
    :returns: markdown text
    """
    url = r"https://api.qrserver.com/v1/create-qr-code/?size=100x100&data="
    txt = f"""<img src={url}"https://arxiv.org/abs/{paper_id}">"""
    txt = '<div id="qrcode">' + txt + '</div>'
    return txt

## get list of arxiv paper candidates

We use the MPIA mitarbeiter list webpage from mpia.de to get author names
We then get all new papers from Arxiv and match authors

In [3]:
# get list from MPIA website
# it automatically filters identified non-scientists :func:`mpia.filter_non_scientists`
mpia_authors = mpia.get_mpia_mitarbeiter_list()
normed_mpia_authors = [k[1] for k in mpia_authors]   # initials + fullname
new_papers = get_new_papers()
# add manual references
add_paper_refs = []
new_papers.extend([get_paper_from_identifier(k) for k in add_paper_refs])

candidates = []
for paperk in new_papers:
    # Check author list with their initials
    normed_author_list = [mpia.get_initials(k) for k in paperk['authors']]
    hl_authors = highlight_authors_in_list(normed_author_list, normed_mpia_authors, verbose=True)
    matches = [(hl, orig) for hl, orig in zip(hl_authors, paperk['authors']) if 'mark' in hl]
    paperk['authors'] = hl_authors
    if matches:
        # only select paper if an author matched our list
        candidates.append(paperk)
print("""Arxiv has {0:,d} new papers today""".format(len(new_papers)))        
print("""          {0:,d} with possible author matches""".format(len(candidates)))

M. Demianenko  ->  M. Demianenko  |  ['M. Demianenko']
J.-U. Pott  ->  J.-U. Pott  |  ['J.-U. Pott']
S. Li  ->  S. Li  |  ['S. Li']
S. Li  ->  S. Li  |  ['S. Li']
J.-U. Pott  ->  J.-U. Pott  |  ['J.-U. Pott']
J. Shi  ->  J. Shi  |  ['J. Shi']
J. Li  ->  J. Li  |  ['J. Li']
Arxiv has 59 new papers today
          5 with possible author matches


# Parse sources and generate relevant outputs

From the candidates, we do the following steps:
* get their tarball from ArXiv (and extract data)
* find the main .tex file: find one with \documentclass{...} (sometimes it's non trivial)
* Check affiliations with :func:`validation`, which uses :func:`mpia.affiliation_verifications`
* If passing the affiliations: we parse the .tex source
   * inject sub-documents into the main (flatten the main document)
   * parse structure, extract information (title, abstract, authors, figures...)
   * handles `\graphicspath` if provided
* Generate the .md document.

In [4]:
documents = []
failed = []
for paper in tqdm(candidates):
    paper_id = paper['identifier'].lower().replace('arxiv:', '')
    
    folder = f'tmp_{paper_id}'

    try:
        if not os.path.isdir(folder):
            folder = retrieve_document_source(f"{paper_id}", f'tmp_{paper_id}')
        
        try:
            doc = latex.LatexDocument(folder, validation=validation)    
        except AffiliationError as affilerror:
            msg = f"ArXiv:{paper_id:s} is not an MPIA paper... " + str(affilerror)
            failed.append((paper, "affiliation error: " + str(affilerror) ))
            continue
        
        # Hack because sometimes author parsing does not work well
        if (len(doc.authors) != len(paper['authors'])):
            doc._authors = paper['authors']
        else:
            # highlight authors (FIXME: doc.highlight_authors)
            # done on arxiv paper already
            doc._authors = highlight_authors_in_list(
                [mpia.get_initials(k) for k in doc.authors], 
                normed_mpia_authors, verbose=True)
        if (doc.abstract) in (None, ''):
            doc._abstract = paper['abstract']
            
        doc.comment = (get_markdown_badge(paper_id) + 
                       "<mark>Appeared on: " + paper['date'] + "</mark> - ")
        if paper['comments']:
            doc.comment += " _" + paper['comments'] + "_"
        
        full_md = doc.generate_markdown_text()
        
        full_md += get_markdown_qrcode(paper_id)
        
        # replace citations
        try:
            bibdata = latex_bib.LatexBib.from_doc(doc)
            full_md = latex_bib.replace_citations(full_md, bibdata)
        except Exception as e:
            print(e)
        
        documents.append((paper_id, full_md))
    except Exception as e:
        warnings.warn(latex.LatexWarning(f"{paper_id:s} did not run properly\n" +
                                         str(e)
                                        ))
        failed.append((paper, "latex error " + str(e)))

  0%|          | 0/5 [00:00<?, ?it/s]

Retrieving document from  https://arxiv.org/e-print/2404.06558


extracting tarball to tmp_2404.06558... done.
Retrieving document from  https://arxiv.org/e-print/2404.06882


extracting tarball to tmp_2404.06882... done.
Retrieving document from  https://arxiv.org/e-print/2404.06893


extracting tarball to tmp_2404.06893... done.
Retrieving document from  https://arxiv.org/e-print/2404.06930


extracting tarball to tmp_2404.06930...

 done.


Found 78 bibliographic references in tmp_2404.06930/paper.bbl.
Retrieving document from  https://arxiv.org/e-print/2404.07149


extracting tarball to tmp_2404.07149...

 done.


### Export the logs

Throughout, we also keep track of the logs per paper. see `logs-{today date}.md` 

In [5]:
import datetime
today = str(datetime.date.today())
logfile = f"_build/html/logs/log-{today}.md"


with open(logfile, 'w') as logs:
    # Success
    logs.write(f'# Arxiv on Deck 2: Logs - {today}\n\n')
    logs.write("""* Arxiv had {0:,d} new papers\n""".format(len(new_papers)))
    logs.write("""    * {0:,d} with possible author matches\n\n""".format(len(candidates)))
    logs.write("## Sucessful papers\n\n")
    display(Markdown("## Successful papers"))
    success = [k[0] for k in documents]
    for candid in candidates:
        if candid['identifier'].split(':')[-1] in success:
            display(candid)
            logs.write(candid.generate_markdown_text() + '\n\n')

    ## failed
    logs.write("## Failed papers\n\n")
    display(Markdown("## Failed papers"))
    failed = sorted(failed, key=lambda x: x[1])
    current_reason = ""
    for paper, reason in failed:
        if 'affiliation' in reason:
            color = 'green'
        else:
            color = 'red'
        data = Markdown(
                paper.generate_markdown_text() + 
                f'\n|<p style="color:{color:s}"> **ERROR** </p>| <p style="color:{color:s}">{reason:s}</p> |'
               )
        if reason != current_reason:
            logs.write(f'### {reason:s} \n\n')
            current_reason = reason
        logs.write(data.data + '\n\n')
        
        # only display here the important errors (all in logs)
        # if color in ('red',):
        display(data)

## Successful papers


|||
|---:|:---|
| [![arXiv](https://img.shields.io/badge/arXiv-arXiv:2404.06930-b31b1b.svg)](https://arxiv.org/abs/arXiv:2404.06930) | **The first spectroscopic IR reverberation programme on Mrk 509**  |
|| J. A. J. Mitchell, et al. -- incl., <mark>J.-U. Pott</mark> |
|*Appeared on*| *2024-04-11*|
|*Comments*| *16 pages, 12 figures*|
|**Abstract**| Near IR spectroscopic reverberation of Active Galactic Nuclei (AGN) potentially allows the infrared (IR) broad line region (BLR) to be reverberated alongside the disc and dust continua, while the spectra can also reveal details of dust astro-chemistry. Here, we describe results of a short pilot study (17 near-IR spectra over a 183 d period) for Mrk 509. The spectra give a luminosity-weighted dust radius of $\langle R_{\mathrm{d,lum}} \rangle = 186 \pm 4$ light-days for blackbody (large grain dust), consistent with previous (photometric) reverberation campaigns, whereas carbon and silicate dust give much larger radii. We develop a method of calibrating spectral data in objects where the narrow lines are extended beyond the slit width. We demonstrate this by showing our resultant photometric band light curves are consistent with previous results, with a hot dust lag at >40 d in the K band, clearly different from the accretion disc response at <20 d in the z band. We place this limit of 40 d by demonstrating clearly that the modest variability that we do detect in the H and K band does not reverberate on time-scales of less than 40 d. We also extract the Pa$\beta$ line light curve, and find a lag which is consistent with the optical BLR H$\beta$ line of $\sim$70-90 d. This is important as direct imaging of the near-IR BLR is now possible in a few objects, so we need to understand its relation to the better studied optical BLR. |

## Failed papers


|||
|---:|:---|
| [![arXiv](https://img.shields.io/badge/arXiv-arXiv:2404.06558-b31b1b.svg)](https://arxiv.org/abs/arXiv:2404.06558) | **PSF quality metrics in the problem of revealing Intermediate-Mass Black  Holes using MICADO@ELT**  |
|| <mark>M. Demianenko</mark>, <mark>J.-U. Pott</mark>, K. Polsterer |
|*Appeared on*| *2024-04-11*|
|*Comments*| *4 pages; submitted to Astronomical Data Analysis Software and Systems (ADASS) XXXIII proceedings*|
|**Abstract**| Nowadays, astronomers perform point spread function (PSF) fitting for most types of observational data. Interpolation of the PSF is often an intermediate step in such algorithms. In the case of the Multi-AO Imaging Camera for Deep Observations (MICADO) at the Extremely Large Telescope (ELT), PSF interpolation will play a crucial role in high-precision astrometry for stellar clusters and confirmation of the Intermediate-Mass Black Holes (IMBHs) presence. Significant PSF variations across the field of view invalidate the approach of deconvolution with a mean PSF or on-axis PSF. The ignoring of PSF variations can be especially unsatisfactory in the case of Single Conjugate Adaptive Optics (SCAO) observations, as these sophisticated and expensive systems are designed to achieve high resolution with ground-based telescopes by correcting for atmospheric turbulence in the direction of one reference star. In plenty of tasks, you face the question: How can I establish the quality of PSF fitting or interpolation? Our study aims to demonstrate the variety of PSF quality metrics, including the problem of revealing IMBHs in stellar clusters. |
|<p style="color:green"> **ERROR** </p>| <p style="color:green">affiliation error: mpia.affiliation_verifications: '69117' keyword not found.</p> |


|||
|---:|:---|
| [![arXiv](https://img.shields.io/badge/arXiv-arXiv:2404.06882-b31b1b.svg)](https://arxiv.org/abs/arXiv:2404.06882) | **A Multi-wavelength Study on A Gamma-ray Bright AGN 1308+326 Using KVN at  22 and 43 GHz**  |
|| <mark>S. Li</mark>, S.-s. Lee, W. Y. Cheong |
|*Appeared on*| *2024-04-11*|
|*Comments*| *16 pages, 7 figures, Accepted for publication in JKAS*|
|**Abstract**| In this paper, we conduct a multi-frequency analysis of the gamma-ray bright blazar 1308+326 from February 2013 to March 2020, using the Korean VLBI Network at 22 and 43 GHz and gamma-ray data from the Fermi Large Area Telescope (LAT). Our findings reveal spectral variations around the 2014 gamma-ray flare, aligning with the shock-in-jet model. A strong correlation is observed between gamma-ray and 43 GHz emissions, with a 27-day lag in the VLBI core light curve, indicating a 50-day delay from the beginning of a specific radio flare to the gamma-ray peak. This radio flare correlates with a new jet component, suggesting the 2014 gamma-ray flare resulted from its interaction with a stationary component. Our analysis indicates the 2014 gamma-ray flare originated 40-63 parsecs from the central engine, with seed photons for the gamma-ray emission unlikely from the broad-line region. |
|<p style="color:green"> **ERROR** </p>| <p style="color:green">affiliation error: mpia.affiliation_verifications: 'Heidelberg' keyword not found.</p> |


|||
|---:|:---|
| [![arXiv](https://img.shields.io/badge/arXiv-arXiv:2404.06893-b31b1b.svg)](https://arxiv.org/abs/arXiv:2404.06893) | **Impact of reflection Comptonization on X-ray reflection spectroscopy:  the case of EXO 1846-031**  |
|| <mark>S. Li</mark>, et al. |
|*Appeared on*| *2024-04-11*|
|*Comments*| *12 pages, 7 figures*|
|**Abstract**| Within the disk-corona model, it is natural to expect that a fraction of reflection photons from the disk are Compton scattered by the hot corona (reflection Comptonization), even if this effect is usually ignored in X-ray reflection spectroscopy studies. We study the effect by using NICER and NuSTAR data of the Galactic black hole EXO 1846-031 in the hard-intermediate state with the model SIMPLCUTX. Our analysis suggests that a scattered fraction of order 10% is required to fit the data, but the inclusion of reflection Comptonization does not change appreciably the measurements of key-parameters like the black hole spin and the inclination angle of the disk. |
|<p style="color:green"> **ERROR** </p>| <p style="color:green">affiliation error: mpia.affiliation_verifications: 'Heidelberg' keyword not found.</p> |


|||
|---:|:---|
| [![arXiv](https://img.shields.io/badge/arXiv-arXiv:2404.07149-b31b1b.svg)](https://arxiv.org/abs/arXiv:2404.07149) | **Tianyu: search for the second solar system and explore the dynamic  universe**  |
|| F. Feng, et al. -- incl., <mark>J. Shi</mark>, <mark>J. Li</mark> |
|*Appeared on*| *2024-04-11*|
|*Comments*| *48 pages, 16 figures, accepted by Acta Astronomica Sinica*|
|**Abstract**| Giant planets like Jupiter and Saturn, play important roles in the formation and habitability of Earth-like planets. The detection of solar system analogs that have multiple cold giant planets is essential for our understanding of planet habitability and planet formation. Although transit surveys such as Kepler and TESS have discovered thousands of exoplanets, these missions are not sensitive to long period planets due to their limited observation baseline. The Tianyu project, comprising two 1-meter telescopes (Tianyu-I and II), is designed to detect transiting cold giant planets in order to find solar system analogs. Featuring a large field of view and equipped with a high-speed CMOS camera, Tianyu-I will perform a high-precision photometric survey of about 100 million stars, measuring light curves at hour-long cadence. The candidates found by Tianyu-I will be confirmed by Tianyu-II and other surveys and follow-up facilities through multi-band photometry, spectroscopy, and high resolution imaging. Tianyu telescopes will be situated at an elevation about 4000 meters in Lenghu, China. With a photometric precision of 1% for stars with V < 18 mag, Tianyu is expected to find more than 300 transiting exoplanets, including about 12 cold giant planets, over five years. A five-year survey of Tianyu would discover 1-2 solar system analogs. Moreover, Tianyu is also designed for non-exoplanetary exploration, incorporating multiple survey modes covering timescales from sub-seconds to months, with a particular emphasis on events occurring within the sub-second to hour range. It excels in observing areas such as infant supernovae, rare variable stars and binaries, tidal disruption events, Be stars, cometary activities, and interstellar objects. These discoveries not only enhance our comprehension of the universe but also offer compelling opportunities for public engagement in scientific exploration. |
|<p style="color:green"> **ERROR** </p>| <p style="color:green">affiliation error: mpia.affiliation_verifications: 'Heidelberg' keyword not found.</p> |

## Export documents

We now write the .md files and export relevant images

In [6]:
def export_markdown_summary(md: str, md_fname:str, directory: str):
    """Export MD document and associated relevant images"""
    import os
    import shutil
    import re

    if (os.path.exists(directory) and not os.path.isdir(directory)):
        raise RuntimeError(f"a non-directory file exists with name {directory:s}")

    if (not os.path.exists(directory)):
        print(f"creating directory {directory:s}")
        os.mkdir(directory)

    fig_fnames = (re.compile(r'\[Fig.*\]\((.*)\)').findall(md) + 
                  re.compile(r'\<img src="([^>\s]*)"[^>]*/>').findall(md))
    for fname in fig_fnames:
        if 'http' in fname:
            # No need to copy online figures
            continue
        destdir = os.path.join(directory, os.path.dirname(fname))
        destfname = os.path.join(destdir, os.path.basename(fname))
        try:
            os.makedirs(destdir)
        except FileExistsError:
            pass
        shutil.copy(fname, destfname)
    with open(os.path.join(directory, md_fname), 'w') as fout:
        fout.write(md)
    print("exported in ", os.path.join(directory, md_fname))
    [print("    + " + os.path.join(directory,fk)) for fk in fig_fnames]

In [7]:
for paper_id, md in documents:
    export_markdown_summary(md, f"{paper_id:s}.md", '_build/html/')

exported in  _build/html/2404.06930.md
    + _build/html/tmp_2404.06930/./images/ztf_lco_corr_full.png
    + _build/html/tmp_2404.06930/./images/siii_flux_window_2panel.png
    + _build/html/tmp_2404.06930/./images/z_band_spec_photom_check.png


## Display the papers

Not necessary but allows for a quick check.

In [8]:
[display(Markdown(k[1])) for k in documents];

<div class="macros" style="visibility:hidden;">
$\newcommand{\ensuremath}{}$
$\newcommand{\xspace}{}$
$\newcommand{\object}[1]{\texttt{#1}}$
$\newcommand{\farcs}{{.}''}$
$\newcommand{\farcm}{{.}'}$
$\newcommand{\arcsec}{''}$
$\newcommand{\arcmin}{'}$
$\newcommand{\ion}[2]{#1#2}$
$\newcommand{\textsc}[1]{\textrm{#1}}$
$\newcommand{\hl}[1]{\textrm{#1}}$
$\newcommand{\footnote}[1]{}$
$\newcommand{\la}{\mathrel{\hbox{\rlap{\hbox{\lower4pt\hbox{\sim}}}\hbox{<}}}}$
$\newcommand{\ga}{\mathrel{\hbox{\rlap{\hbox{\lower4pt\hbox{\sim}}}\hbox{>}}}}$
$\newcommand{\OIII}{[O~{\sevenrm III}]}$
$\newcommand{\FeII}{Fe~{\sevenrm II}}$
$\newcommand{\FeIIf}{[Fe~{\sevenrm II}]}$
$\newcommand{\SIII}{[S~{\sevenrm III}]}$
$\newcommand{\HeI}{He~{\sevenrm I}}$
$\newcommand{\HeII}{He~{\sevenrm II}}$
$\newcommand{\NeV}{[Ne~{\sevenrm V}]}$
$\newcommand{\OIV}{[O~{\sevenrm IV}]}$
$\newcommand{\iraf}{{\sevenrm IRAF}}$
$\newcommand{\mpfit}{{\sevenrm MPFIT}}$
$\newcommand{\galfit}{{\sevenrm GALFIT}}$
$\newcommand{\mapspec}{{\sevenrm mapspec}}$
$\newcommand{\cream}{{\sevenrm CREAM}}$
$\newcommand{\javelin}{{\sevenrm JAVELIN}}$
$\newcommand{\clumpy}{{\sevenrm CLUMPY}}$
$\newcommand{\cloudy}{{\sevenrm CLOUDY}}$
$\newcommand{\astroimagej}{{\sevenrm AstroImageJ}}$
$\newcommand{\banzai}{{\sevenrm BANZAI}}$
$\newcommand{\orac}{{\sevenrm ORAC}}$
$\newcommand{\python}{{\sevenrm python}}$
$\newcommand{\prepspec}{{\sevenrm prepspec}}$
$\newcommand{\GW}{{\sevenrm GW92}}$
$\newcommand{\spextool}{{\sevenrm Spextool}}$</div>



<div id="title">

# The first spectroscopic IR reverberation programme on Mrk 509

</div>
<div id="comments">

[![arXiv](https://img.shields.io/badge/arXiv-2404.06930-b31b1b.svg)](https://arxiv.org/abs/2404.06930)<mark>Appeared on: 2024-04-11</mark> -  _16 pages, 12 figures_

</div>
<div id="authors">

J. A. J. Mitchell, et al. -- incl., <mark>J.-U. Pott</mark>

</div>
<div id="abstract">

**Abstract:** Near IR spectroscopic reverberation of Active Galactic Nuclei (AGN) potentiallyallows the IR broad line region (BLR)to be reverberated alongside the disc and dust continua, while the spectra can also reveal details of dust astro-chemistry. Here we describe results of a short pilot study (17 near-IR spectra over a 183 day period) for Mrk 509. The spectra give a luminosity-weighted dust radius of $\mbox{$\langle R_{\rm d,lum} \rangle = 186\pm4$ light-days}$ for blackbody (large grain dust), consistent with previous (photometric) reverberation campaigns, whereas carbon and silicate dust give much larger radii.We develop a method of calibrating spectral data in objects where the narrow lines are extended beyond the slit width. Wedemonstrate this by showing our resultantphotometric band lightcurves are consistent with previous results, with ahot dust lag at >40 days in the $K$ -band, clearly different from the accretion disc response at <20 days in the $z$ -band. We place this limit of 40 days by demonstrating clearly that the modest variability that we do detect in the $H$ and $K$ -band does not reverberate on timescales of less than 40 days.We also extract the Pa $\beta$ line lightcurve, and find a lag which is consistent with the optical BLR H $\beta$ line of $\sim70-90$ days. This is important as direct imaging of the near-IR BLR is now possible in a few objects, so we need to understand its relation to the better studied optical BLR.

</div>

<div id="div_fig1">

<img src="tmp_2404.06930/./images/ztf_lco_corr_full.png" alt="Fig9" width="100%"/>

**Figure 9. -**  LCO $g$-band observations of Mrk 509 with complementary corrected ZTF $g$ band data. The most significant variability feature measured contemporaneously with our near-IR spectra is shaded in red. Archival LCOGT data (MJD<58200) is shown to extend the time baseline. Spectral epochs are marked by red arrows.  (*ztf_lco*)

</div>
<div id="div_fig2">

<img src="tmp_2404.06930/./images/siii_flux_window_2panel.png" alt="Fig2" width="100%"/>

**Figure 2. -** **Top Panel:**  Pa$\beta$ emission line from Mrk 509 spectrum taken on 26 August 2019. Red line represents a linearly fitted continuum, subtracted in order to calculate enclosed flux. The continuum has been fit using a blueward clean continuum window of (12500-12640) Å  and the redward clean continuum window of (13000-13050) Å, these regions are highlighted in blue. The redward window is necessarily short as beyond this there is significant telluric absorption. **Bottom Panel:**  Mean spectrum of Mrk 509 from all 4 nights with photometric conditions. Pa$\mathrm{\beta}$ profile (black) scaled to match Pa$\mathrm{\epsilon}$(red), in order to isolate the constant $\SIII$$\mathrm{$\la$mbda 9531}$ component from any variable broad line emission. The slope of the Pa$\mathrm{\beta}$ profile in the $\SIII$  window is shown as the artificial continuum in grey. Both panels centre on the rest frame wavelengths of 12821.6 $\mathrm{\Angstrom}$ and 9548.6 $\mathrm{\Angstrom}$ for Pa$\beta$ and Pa$\epsilon$ respectively and have a matched velocity range.
 (*siii_flux_window*)

</div>
<div id="div_fig3">

<img src="tmp_2404.06930/./images/z_band_spec_photom_check.png" alt="Fig4" width="100%"/>

**Figure 4. -** **Top Panel:** LCO $z_s$ band photometry, uncorrected for host galaxy contribution in black, with all 17 spectral LCO $z_s$ band (8180-9220 $\mathrm{\Angstrom}$ observed frame) uncorrected fluxes plotted in red. **Second Panel:** LCO $z_s$ band photometry, uncorrected for host galaxy contribution in black, with all 17 spectral  LCO $z_s$ band ((8180-9220) $\mathrm{\Angstrom}$ observed frame) fluxes, corrected with $\mapspec$  scaling factors are plotted in red.**Third Panel:** LCO $z_s$ band photometry, corrected for a constant host galaxy contribution in black, with all 17 spectral LCO $z_s$ band (8180-9220 $\mathrm{\Angstrom}$ observed frame) fluxes, corrected with the $\mapspec$  scaling factors  plotted in red. Spectral fluxes measured on MJD 58745.30 and 58801.20 are encircled in red as they do not match well with the   LCO $z_s$ band photometry. The night of 26$\mathrm{^{th}}$ August 2019 is highlighted with a dashed red line. **Bottom Panel:** The $\mapspec$  corrected lightcurve has been further corrected to the LCO $z_s$ band photometry by scaling to the weighted mean of all photometric data points within three days of each spectral data point. The errors displayed are defined by the spread of the data to which the spectral points have been corrected, or the mean spread of the photometry within any given 3 day period of the spread if the correction was less than this value. These uncertainties have been propagated to our corrected lightcurves along with the $\mapspec$  uncertainties displayed in the panel above.   (*z_check*)

</div><div id="qrcode"><img src=https://api.qrserver.com/v1/create-qr-code/?size=100x100&data="https://arxiv.org/abs/2404.06930"></div>

# Create HTML index

In [9]:
from datetime import datetime, timedelta, timezone
from glob import glob
import os

files = glob('_build/html/*.md')
days = 7
now = datetime.today()
res = []
for fk in files:
    stat_result = os.stat(fk).st_ctime
    modified = datetime.fromtimestamp(stat_result, tz=timezone.utc).replace(tzinfo=None)
    delta = now.today() - modified
    if delta <= timedelta(days=days):
        res.append((delta.seconds, fk))
res = [k[1] for k in reversed(sorted(res, key=lambda x:x[1]))]
npub = len(res)
print(len(res), f" publications files modified in the last {days:d} days.")
# [ print('\t', k) for k in res ];

401  publications files modified in the last 7 days.


In [10]:
import datetime
from glob import glob

def get_last_n_days(lst, days=1):
    """ Get the documents from the last n days """
    sorted_lst = sorted(lst, key=lambda x: x[1], reverse=True)
    for fname, date in sorted_lst:
        if date >= str(datetime.date.today() - datetime.timedelta(days=days)):
            yield fname

def extract_appearance_dates(lst_file):
    dates = []

    def get_date(line):
        return line\
            .split('Appeared on:')[-1]\
            .split('</mark>')[0].strip()

    for fname in lst:
        with open(fname, 'r') as f:
            found_date = False
            for line in f:
                if not found_date:
                    if "Appeared on" in line:
                        found_date = True
                        dates.append((fname, get_date(line)))
                else:
                    break
    return dates

from glob import glob
lst = glob('_build/html/*md')
days = 7
dates = extract_appearance_dates(lst)
res = list(get_last_n_days(dates, days))
npub = len(res)
print(len(res), f" publications in the last {days:d} days.")

7  publications in the last 7 days.


In [11]:
def create_carousel(npub=4):
    """ Generate the HTML code for a carousel with `npub` slides """
    carousel = ["""  <div class="carousel" """,
                """       data-flickity='{ "autoPlay": 10000, "adaptiveHeight": true, "resize": true, "wrapAround": true, "pauseAutoPlayOnHover": true, "groupCells": 1 }' id="asyncTypeset">"""
                ]
    
    item_str = """    <div class="carousel-cell"> <div id="slide{k}" class="md_view">Content {k}</div> </div>"""
    for k in range(1, npub + 1):
        carousel.append(item_str.format(k=k))
    carousel.append("  </div>")
    return '\n'.join(carousel)

def create_grid(npub=4):
    """ Generate the HTML code for a flat grid with `npub` slides """
    grid = ["""  <div class="grid"> """,
                ]
    
    item_str = """    <div class="grid-item"> <div id="slide{k}" class="md_view">Content {k}</div> </div>"""
    for k in range(1, npub + 1):
        grid.append(item_str.format(k=k))
    grid.append("  </div>")
    return '\n'.join(grid)

In [12]:
carousel = create_carousel(npub)
docs = ', '.join(['"{0:s}"'.format(k.split('/')[-1]) for k in res])
slides = ', '.join([f'"slide{k}"' for k in range(1, npub + 1)])

with open("daily_template.html", "r") as tpl:
    page = tpl.read()
    page = page.replace("{%-- carousel:s --%}", carousel)\
               .replace("{%-- suptitle:s --%}",  "7-day archives" )\
               .replace("{%-- docs:s --%}", docs)\
               .replace("{%-- slides:s --%}", slides)
    
with open("_build/html/index_7days.html", 'w') as fout:
    fout.write(page)

In [13]:
# redo for today
days = 1
res = list(get_last_n_days(dates, days))
npub = len(res)
print(len(res), f" publications in the last day.")

carousel = create_carousel(npub)
docs = ', '.join(['"{0:s}"'.format(k.split('/')[-1]) for k in res])
slides = ', '.join([f'"slide{k}"' for k in range(1, npub + 1)])

with open("daily_template.html", "r") as tpl:
    page = tpl.read()
    page = page.replace("{%-- carousel:s --%}", carousel)\
               .replace("{%-- suptitle:s --%}",  "Daily" )\
               .replace("{%-- docs:s --%}", docs)\
               .replace("{%-- slides:s --%}", slides)
    
# print(carousel, docs, slides)
# print(page)
with open("_build/html/index_daily.html", 'w') as fout:
    fout.write(page)

2  publications in the last day.


In [14]:
# Create the flat grid of the last N papers (fixed number regardless of dates)
from itertools import islice 

npub = 6
res = [k[0] for k in (islice(reversed(sorted(dates, key=lambda x: x[1])), 6))]
print(len(res), f" {npub} publications selected.")

grid = create_grid(npub)
docs = ', '.join(['"{0:s}"'.format(k.split('/')[-1]) for k in res])
slides = ', '.join([f'"slide{k}"' for k in range(1, npub + 1)])

with open("grid_template.html", "r") as tpl:
    page = tpl.read()
    page = page.replace("{%-- grid-content:s --%}", grid)\
               .replace("{%-- suptitle:s --%}",  f"Last {npub:,d} papers" )\
               .replace("{%-- docs:s --%}", docs)\
               .replace("{%-- slides:s --%}", slides)
    
# print(grid, docs, slides)
# print(page)
with open("_build/html/index_npub_grid.html", 'w') as fout:
    fout.write(page)

6  6 publications selected.
